In [ ]:
# Montar Google Drive David Sanzana entorno ejecución t4gpu
from google.colab import drive
drive.mount('/content/drive')

# Instalar datasets si es necesario
!pip install datasets transformers torch scikit-learn pandas

# Importar bibliotecas necesarias
import os
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Lista de tácticas
tacticas = ["blowfish", "digest authentication", "input validation",
           "kerberos", "ldap", "md5", "oauth2", "rsa",
           "session management", "sftp", "sha256", "sha512",
           "ssh", "vpn", "acl", "aes", "audit trail",
           "3des", "tls", "token based authentication"]

# Función para cargar datasets con manejo de clases unrelated
def cargar_datasets_desde_drive(tacticas):
    datasets_dict = {}
    unrelated_samples = []

    for tactica in tacticas:
        ruta_archivo = f'/content/drive/MyDrive/datasets/{tactica.replace(" ", "_")}.csv'

        try:
            df = pd.read_csv(ruta_archivo)

            # Separar ejemplos unrelated
            df_relacionados = df[df['class'] == tactica]
            df_unrelated = df[df['class'] != tactica]

            # Agregar ejemplos relacionados al diccionario de datasets
            if not df_relacionados.empty:
                datasets_dict[tactica] = Dataset.from_pandas(df_relacionados)

            # Recopilar ejemplos unrelated
            if not df_unrelated.empty:
                df_unrelated['class'] = 'unrelated'
                unrelated_samples.append(df_unrelated)

        except FileNotFoundError:
            print(f"Archivo no encontrado para {tactica}")

    # Combinar ejemplos unrelated
    if unrelated_samples:
        df_unrelated_final = pd.concat(unrelated_samples, ignore_index=True)
        datasets_dict['unrelated'] = Dataset.from_pandas(df_unrelated_final)

    return datasets_dict

# Cargar datasets para cada táctica
datasets = cargar_datasets_desde_drive(tacticas)

# Unir todos los datasets en uno solo
datos_combinados = DatasetDict({
    "entrenamiento": Dataset.from_pandas(
        pd.concat(
            [datasets[tactica].to_pandas() for tactica in datasets.keys()],
            ignore_index=True  # Evitar conflictos de índices
        )
    )
})

# Inicializar el tokenizador y el modelo de CodeBERT
# Añadir 1 para la clase unrelated
tokenizador = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
modelo = RobertaForSequenceClassification.from_pretrained(
    "microsoft/codebert-base",
    num_labels=len(tacticas) + 1  # Añadir 1 para unrelated
)

# Tokenizar los datos
def tokenizar_datos(ejemplos):
    return tokenizador(ejemplos['text'], truncation=True, padding='max_length')

# Modificar la función de etiquetas para manejar 'unrelated'
def agregar_etiquetas(ejemplos):
    def obtener_indice_etiqueta(clase):
        return len(tacticas) if clase == 'unrelated' else tacticas.index(clase)

    etiquetas = [obtener_indice_etiqueta(clase) for clase in ejemplos['class']]
    ejemplos['labels'] = etiquetas
    return ejemplos

# Preprocesar datos de entrenamiento
datos_entrenamiento = datos_combinados['entrenamiento'].map(tokenizar_datos, batched=True)
datos_entrenamiento = datos_entrenamiento.map(agregar_etiquetas, batched=True)

# Función para calcular las métricas
def calcular_metricas(prediccion):
    etiquetas = prediccion.label_ids
    predicciones = prediccion.predictions.argmax(-1)
    aciertos = (predicciones == etiquetas).mean()
    precision = precision_score(etiquetas, predicciones, average='weighted')
    recall = recall_score(etiquetas, predicciones, average='weighted')
    f1 = f1_score(etiquetas, predicciones, average='weighted')
    return {
        'aciertos': aciertos,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Función para realizar la validación cruzada
def validacion_cruzada_modelo(modelo, datos_combinados, tokenizador, num_pliegues=5):
    X = datos_entrenamiento['text']  # Los textos de los fragmentos de código
    y = datos_entrenamiento['labels']  # Las etiquetas numéricas correspondientes a las tácticas
    #
    skf = StratifiedKFold(n_splits=num_pliegues, shuffle=True, random_state=42)
    aciertos_por_pliegue = []
    precision_por_pliegue = []
    recall_por_pliegue = []
    f1_por_pliegue = []

    for pliegue, (entrenamiento_idx, validacion_idx) in enumerate(skf.split(X, y)):
        print(f"--- Pliegue {pliegue + 1} ---")

        # Obtener los datos de entrenamiento y validación para este pliegue
        dataset_entrenamiento = datos_entrenamiento.select(entrenamiento_idx)
        dataset_validacion = datos_entrenamiento.select(validacion_idx)

        # Configurar el entrenamiento y asigna epocas
        argumentos_entrenamiento = TrainingArguments(
            output_dir="./resultados",
            num_train_epochs=5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            warmup_steps=500,
            weight_decay=0.01,
            logging_dir="./logs",
            logging_steps=10,
            evaluation_strategy="epoch"
        )

        # Crear el entrenador para este pliegue
        entrenador = Trainer(
            model=modelo,
            args=argumentos_entrenamiento,
            train_dataset=dataset_entrenamiento,
            eval_dataset=dataset_validacion,
            compute_metrics=calcular_metricas
        )

        # Entrenar el modelo en este pliegue
        entrenador.train()


# Realizar la validación cruzada
validacion_cruzada_modelo(modelo, datos_entrenamiento, tokenizador, num_pliegues=5)



# Guardar el modelo entrenado
def guardar_modelo(modelo, ruta_guardado):
    torch.save(modelo.state_dict(), ruta_guardado)
    print(f"Modelo guardado en {ruta_guardado}")

# Ruta para guardar el modelo
ruta_modelo_guardado = '/content/drive/MyDrive/entrenado/modelo_entrenado.pth'

# Guardamos el modelo
guardar_modelo(modelo, ruta_modelo_guardado)

In [ ]:
import os
import torch
from torch.nn.functional import softmax
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Lista de tácticas
tacticas = ["blowfish", "digest authentication", "input validation",
            "kerberos", "ldap", "md5", "oauth2", "rsa",
            "session management", "sftp", "sha256", "sha512",
            "ssh", "vpn", "acl", "aes", "audit trail",
            "3des", "tls", "token based authentication"]

# Cargar el modelo entrenado
def cargar_modelo(ruta_modelo):
    modelo = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', num_labels=len(tacticas) + 1)  # Añadir 1 para 'unrelated'
    modelo.load_state_dict(torch.load(ruta_modelo))
    modelo.eval()  # Coloca el modelo en modo evaluación
    return modelo

# Cargar el modelo
ruta_modelo_guardado = '/content/drive/MyDrive/entrenado/modelo_entrenado.pth'  # Ajusta la ruta si es necesario
modelo = cargar_modelo(ruta_modelo_guardado)

# Asegúrate de cargar el tokenizador correspondiente
tokenizador = RobertaTokenizer.from_pretrained('microsoft/codebert-base')  # Usa el nombre de tu modelo base

def clasificar_codigo_con_probabilidad(fragmento, modelo, tokenizador, tacticas):
    # Tokenización del fragmento de código
    entradas = tokenizador(fragmento, return_tensors="pt", truncation=True, padding=True)
    entradas = entradas.to(modelo.device)  # Mover las entradas al dispositivo correcto (GPU o CPU)

    # Hacer la predicción
    salidas = modelo(**entradas)  # Realizar la predicción con el modelo
    logits = salidas.logits  # Obtener los logits de la salida del modelo

    # Calcular las probabilidades utilizando softmax
    probabilidades = softmax(logits, dim=-1)

    # Obtener la clase predicha (con la mayor probabilidad)
    clase_predicha = torch.argmax(probabilidades, dim=-1).item()

    # Obtener la probabilidad de la clase predicha
    probabilidad_predicha = probabilidades[0, clase_predicha].item()

    # Retornar la táctica predicha y la probabilidad
    return tacticas[clase_predicha] if clase_predicha < len(tacticas) else 'unrelated', probabilidad_predicha

# Ruta del proyecto
ruta_proyecto = '/content/drive/MyDrive/proyectos/cp2k-master'  # Ajusta la ruta del proyecto

# Leer archivos de la carpeta del proyecto
def leer_archivos_de_directorio(ruta_directorio):
    fragmentos_codigo = []
    rutas_archivos = []
    for root, _, archivos in os.walk(ruta_directorio):
        for archivo in archivos:
            if archivo.endswith(('.py', '.js', '.java', '.cpp', '.f', '.for', '.f90', '.f95', '.f03', '.f08', '.c', '.sh', '.bb', '.bbappend', '.bbclass', '.cmake', '.ipynb', '.cu', '.cuh', '.bas', '.vb', '.vbs', '.cc', '.cxx', '.c++', '.h', '.hpp', '.hh', '.hxx')):  # Filtrar extensiones
                ruta_archivo = os.path.join(root, archivo)
                with open(ruta_archivo, 'r', encoding='utf-8') as f:
                    codigo = f.read()
                    fragmentos_codigo.append(codigo)
                    rutas_archivos.append(ruta_archivo)
    return fragmentos_codigo, rutas_archivos

# Leer y clasificar los archivos del proyecto
fragmentos_codigo, rutas_archivos = leer_archivos_de_directorio(ruta_proyecto)

# Clasificar cada fragmento de código y mostrar la probabilidad
for fragmento, ruta_archivo in zip(fragmentos_codigo, rutas_archivos):
    tactica_predicha, probabilidad_predicha = clasificar_codigo_con_probabilidad(fragmento, modelo, tokenizador, tacticas)
    print(f"Archivo: {ruta_archivo}\nTáctica predicha: {tactica_predicha}\nProbabilidad de predicción: {probabilidad_predicha:.4f}\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-9e3bbd4d05ca>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explici

Archivo: /content/drive/MyDrive/proyectos/cp2k-master/tools/diff_cp2k.py
Táctica predicha: unrelated
Probabilidad de predicción: 0.9976

Archivo: /content/drive/MyDrive/proyectos/cp2k-master/tools/fix_unused_public.py
Táctica predicha: unrelated
Probabilidad de predicción: 0.9970

Archivo: /content/drive/MyDrive/proyectos/cp2k-master/tools/trajana.f90
Táctica predicha: unrelated
Probabilidad de predicción: 0.9993

Archivo: /content/drive/MyDrive/proyectos/cp2k-master/tools/pre-commit-install.sh
Táctica predicha: ssh
Probabilidad de predicción: 0.9695

Archivo: /content/drive/MyDrive/proyectos/cp2k-master/tools/conventions/test_conventions.sh
Táctica predicha: unrelated
Probabilidad de predicción: 0.9983

Archivo: /content/drive/MyDrive/proyectos/cp2k-master/tools/conventions/analyze_gfortran_warnings.py
Táctica predicha: unrelated
Probabilidad de predicción: 0.9956

Archivo: /content/drive/MyDrive/proyectos/cp2k-master/tools/conventions/analyze_gfortran_ast.py
Táctica predicha: unrelat

In [ ]:
import os
import torch
from torch.nn.functional import softmax
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Lista de tácticas
tacticas = ["blowfish", "digest authentication", "input validation",
            "kerberos", "ldap", "md5", "oauth2", "rsa",
            "session management", "sftp", "sha256", "sha512",
            "ssh", "vpn", "acl", "aes", "audit trail",
            "3des", "tls", "token based authentication"]

# Cargar el modelo entrenado
def cargar_modelo(ruta_modelo):
    modelo = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', num_labels=len(tacticas) + 1)  # Añadir 1 para 'unrelated'
    modelo.load_state_dict(torch.load(ruta_modelo))
    modelo.eval()  # Coloca el modelo en modo evaluación
    return modelo

# Cargar el modelo
ruta_modelo_guardado = '/content/drive/MyDrive/entrenado/modelo_entrenado.pth'  # Ajusta la ruta si es necesario
modelo = cargar_modelo(ruta_modelo_guardado)

# Asegúrate de cargar el tokenizador correspondiente
tokenizador = RobertaTokenizer.from_pretrained('microsoft/codebert-base')  # Usa el nombre de tu modelo base

def clasificar_codigo_con_probabilidad(fragmento, modelo, tokenizador, tacticas):
    # Tokenización del fragmento de código
    entradas = tokenizador(fragmento, return_tensors="pt", truncation=True, padding=True)
    entradas = entradas.to(modelo.device)  # Mover las entradas al dispositivo correcto (GPU o CPU)

    # Hacer la predicción
    salidas = modelo(**entradas)  # Realizar la predicción con el modelo
    logits = salidas.logits  # Obtener los logits de la salida del modelo

    # Calcular las probabilidades utilizando softmax
    probabilidades = softmax(logits, dim=-1)

    # Obtener la clase predicha (con la mayor probabilidad)
    clase_predicha = torch.argmax(probabilidades, dim=-1).item()

    # Obtener la probabilidad de la clase predicha
    probabilidad_predicha = probabilidades[0, clase_predicha].item()

    # Retornar la táctica predicha y la probabilidad
    return tacticas[clase_predicha] if clase_predicha < len(tacticas) else 'unrelated', probabilidad_predicha

# Ruta del proyecto
ruta_proyecto = '/content/drive/MyDrive/proyectos/SilQ-master'  # Ajusta la ruta del proyecto

# Leer archivos de la carpeta del proyecto
def leer_archivos_de_directorio(ruta_directorio):
    fragmentos_codigo = []
    rutas_archivos = []
    for root, _, archivos in os.walk(ruta_directorio):
        for archivo in archivos:
            if archivo.endswith(('.py', '.js', '.java', '.cpp', '.f', '.for', '.f90', '.f95', '.f03', '.f08', '.c', '.sh', '.bb', '.bbappend', '.bbclass', '.cmake', '.ipynb', '.cu', '.cuh', '.bas', '.vb', '.vbs', '.cc', '.cxx', '.c++', '.h', '.hpp', '.hh', '.hxx')):  # Filtrar extensiones
                ruta_archivo = os.path.join(root, archivo)
                with open(ruta_archivo, 'r', encoding='utf-8') as f:
                    codigo = f.read()
                    fragmentos_codigo.append(codigo)
                    rutas_archivos.append(ruta_archivo)
    return fragmentos_codigo, rutas_archivos

# Leer y clasificar los archivos del proyecto
fragmentos_codigo, rutas_archivos = leer_archivos_de_directorio(ruta_proyecto)

# Clasificar cada fragmento de código y mostrar la probabilidad
for fragmento, ruta_archivo in zip(fragmentos_codigo, rutas_archivos):
    tactica_predicha, probabilidad_predicha = clasificar_codigo_con_probabilidad(fragmento, modelo, tokenizador, tacticas)
    print(f"Archivo: {ruta_archivo}\nTáctica predicha: {tactica_predicha}\nProbabilidad de predicción: {probabilidad_predicha:.4f}\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-6-dbd4da84ba09>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explici

Archivo: /content/drive/MyDrive/proyectos/SilQ-master/setup.py
Táctica predicha: unrelated
Probabilidad de predicción: 0.9769

Archivo: /content/drive/MyDrive/proyectos/SilQ-master/silq/version.py
Táctica predicha: unrelated
Probabilidad de predicción: 0.9169

Archivo: /content/drive/MyDrive/proyectos/SilQ-master/silq/__init__.py
Táctica predicha: unrelated
Probabilidad de predicción: 0.9981

Archivo: /content/drive/MyDrive/proyectos/SilQ-master/silq/tests/test_suite.py
Táctica predicha: unrelated
Probabilidad de predicción: 0.9810

Archivo: /content/drive/MyDrive/proyectos/SilQ-master/silq/tests/test_config.py
Táctica predicha: unrelated
Probabilidad de predicción: 0.9961

Archivo: /content/drive/MyDrive/proyectos/SilQ-master/silq/tests/test_instruments.py
Táctica predicha: unrelated
Probabilidad de predicción: 0.9991

Archivo: /content/drive/MyDrive/proyectos/SilQ-master/silq/tests/test_analysis.py
Táctica predicha: unrelated
Probabilidad de predicción: 0.9992

Archivo: /content/driv

In [ ]:
import os
import torch
from torch.nn.functional import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

tacticas = ["blowfish", "digest authentication", "input validation",
           "kerberos", "ldap", "md5", "oauth2", "rsa",
           "session management", "sftp", "sha256", "sha512",
           "ssh", "vpn", "acl", "aes", "audit trail",
           "3des", "tls", "token based authentication"]

# Cargar el modelo entrenado
def cargar_modelo(ruta_modelo):
    modelo = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', num_labels=len(tacticas) + 1)  # Añadir 1 para 'unrelated'
    modelo.load_state_dict(torch.load(ruta_modelo))
    modelo.eval()  # Coloca el modelo en modo evaluación
    return modelo

# Cargar el modelo
ruta_modelo_guardado = '/content/drive/MyDrive/entrenado/modelo_entrenado.pth'  # Ajusta la ruta si es necesario
modelo = cargar_modelo(ruta_modelo_guardado)

# Asegúrate de cargar el tokenizador correspondiente
tokenizador = AutoTokenizer.from_pretrained('microsoft/codebert-base')  # Usa el nombre de tu modelo base

def clasificar_codigo_con_probabilidad(fragmento, modelo, tokenizador, tacticas):
    # Tokenización del fragmento de código
    entradas = tokenizador(fragmento, return_tensors="pt", truncation=True, padding=True)
    entradas = entradas.to(modelo.device)  # Mover las entradas al dispositivo correcto (GPU o CPU)

    # Hacer la predicción
    salidas = modelo(**entradas)  # Realizar la predicción con el modelo
    logits = salidas.logits  # Obtener los logits de la salida del modelo

    # Calcular las probabilidades utilizando softmax
    probabilidades = softmax(logits, dim=-1)

    # Obtener la clase predicha (con la mayor probabilidad)
    clase_predicha = torch.argmax(probabilidades, dim=-1).item()

    # Obtener la probabilidad de la clase predicha
    probabilidad_predicha = probabilidades[0, clase_predicha].item()

    # Retornar la táctica predicha y la probabilidad
    return tacticas[clase_predicha] if clase_predicha < len(tacticas) else 'unrelated', probabilidad_predicha

# Ruta del proyecto
ruta_proyecto = '/content/drive/MyDrive/proyectos/QUICK-master'  # Ajusta la ruta del proyecto

# Leer archivos de la carpeta del proyecto
# Leer archivos de la carpeta del proyecto
def leer_archivos_de_directorio(ruta_directorio):
    fragmentos_codigo = []
    rutas_archivos = []
    for root, _, archivos in os.walk(ruta_directorio):
        for archivo in archivos:
            if archivo.endswith(('.py', '.js', '.java', '.cpp', '.f', '.for', '.f90', '.f95', '.f03', '.f08', '.c', '.sh', '.bb', '.bbappend', '.bbclass', '.cmake', '.ipynb', '.cu', '.cuh', '.bas', '.vb', '.vbs', '.cc', '.cxx', '.c++', '.h', '.hpp', '.hh', '.hxx')):
                ruta_archivo = os.path.join(root, archivo)
                try:
                    # Intenta abrir con utf-8 primero
                    with open(ruta_archivo, 'r', encoding='utf-8') as f:
                        codigo = f.read()
                except UnicodeDecodeError:
                    try:
                        # Si falla, intenta con 'latin-1'
                        with open(ruta_archivo, 'r', encoding='latin-1') as f:
                            codigo = f.read()
                    except UnicodeDecodeError:
                        # Si aún falla, omite el archivo e imprime un mensaje de advertencia
                        print(f"Advertencia: No se pudo decodificar el archivo {ruta_archivo}. Omitiendo...")
                        continue  # Salta al siguiente archivo

                fragmentos_codigo.append(codigo)
                rutas_archivos.append(ruta_archivo)
    return fragmentos_codigo, rutas_archivos

# Leer y clasificar los archivos del proyecto
fragmentos_codigo, rutas_archivos = leer_archivos_de_directorio(ruta_proyecto)

# Clasificar cada fragmento de código y mostrar la probabilidad
for fragmento, ruta_archivo in zip(fragmentos_codigo, rutas_archivos):
    tactica_predicha, probabilidad_predicha = clasificar_codigo_con_probabilidad(fragmento, modelo, tokenizador, tacticas)
    print(f"Archivo: {ruta_archivo}\nTáctica predicha: {tactica_predicha}\nProbabilidad de predicción: {probabilidad_predicha:.4f}\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-cab9be23ca8f>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explici

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Se han truncado las últimas 5000 líneas del flujo de salida.
Archivo: /content/drive/MyDrive/proyectos/QUICK-master/src/lapack/dsptrs.f
Táctica predicha: aes
Probabilidad de predicción: 0.9019

Archivo: /content/drive/MyDrive/proyectos/QUICK-master/src/lapack/dlasd8.f
Táctica predicha: unrelated
Probabilidad de predicción: 0.9783

Archivo: /content/drive/MyDrive/proyectos/QUICK-master/src/lapack/dlacpy.f
Táctica predicha: unrelated
Probabilidad de predicción: 0.9900

Archivo: /content/drive/MyDrive/proyectos/QUICK-master/src/lapack/dlarf.f
Táctica predicha: unrelated
Probabilidad de predicción: 0.9921

Archivo: /content/drive/MyDrive/proyectos/QUICK-master/src/lapack/dtrexc.f
Táctica predicha: unrelated
Probabilidad de predicción: 0.9867

Archivo: /content/drive/MyDrive/proyectos/QUICK-master/src/lapack/dlaset.f
Táctica predicha: unrelated
Probabilidad de predicción: 0.9909

Archivo: /content/drive/MyDrive/proyectos/QUICK-master/src/lapack/dlansy.f
Táctica predicha: unrelated
Probabili

In [ ]:
import os
import torch
from torch.nn.functional import softmax
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Lista de tácticas
tacticas = ["blowfish", "digest authentication", "input validation",
            "kerberos", "ldap", "md5", "oauth2", "rsa",
            "session management", "sftp", "sha256", "sha512",
            "ssh", "vpn", "acl", "aes", "audit trail",
            "3des", "tls", "token based authentication"]

# Cargar el modelo entrenado
def cargar_modelo(ruta_modelo):
    modelo = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', num_labels=len(tacticas) + 1)  # Añadir 1 para 'unrelated'
    modelo.load_state_dict(torch.load(ruta_modelo))
    modelo.eval()  # Coloca el modelo en modo evaluación
    return modelo

# Cargar el modelo
ruta_modelo_guardado = '/content/drive/MyDrive/entrenado/modelo_entrenado.pth'  # Ajusta la ruta si es necesario
modelo = cargar_modelo(ruta_modelo_guardado)

# Asegúrate de cargar el tokenizador correspondiente
tokenizador = RobertaTokenizer.from_pretrained('microsoft/codebert-base')  # Usa el nombre de tu modelo base

def clasificar_codigo_con_probabilidad(fragmento, modelo, tokenizador, tacticas):
    # Tokenización del fragmento de código
    entradas = tokenizador(fragmento, return_tensors="pt", truncation=True, padding=True)
    entradas = entradas.to(modelo.device)  # Mover las entradas al dispositivo correcto (GPU o CPU)

    # Hacer la predicción
    salidas = modelo(**entradas)  # Realizar la predicción con el modelo
    logits = salidas.logits  # Obtener los logits de la salida del modelo

    # Calcular las probabilidades utilizando softmax
    probabilidades = softmax(logits, dim=-1)

    # Obtener la clase predicha (con la mayor probabilidad)
    clase_predicha = torch.argmax(probabilidades, dim=-1).item()

    # Obtener la probabilidad de la clase predicha
    probabilidad_predicha = probabilidades[0, clase_predicha].item()

    # Retornar la táctica predicha y la probabilidad
    return tacticas[clase_predicha] if clase_predicha < len(tacticas) else 'unrelated', probabilidad_predicha

# Ruta del proyecto
ruta_proyecto = '/content/drive/MyDrive/proyectos/piggymetrics-master'  # Ajusta la ruta del proyecto

# Leer archivos de la carpeta del proyecto
def leer_archivos_de_directorio(ruta_directorio):
    fragmentos_codigo = []
    rutas_archivos = []
    for root, _, archivos in os.walk(ruta_directorio):
        for archivo in archivos:
            if archivo.endswith(('.py', '.js', '.java', '.cpp', '.f', '.for', '.f90', '.f95', '.f03', '.f08', '.c', '.sh', '.bb', '.bbappend', '.bbclass', '.cmake', '.ipynb', '.cu', '.cuh', '.bas', '.vb', '.vbs', '.cc', '.cxx', '.c++', '.h', '.hpp', '.hh', '.hxx')):  # Filtrar extensiones
                ruta_archivo = os.path.join(root, archivo)
                with open(ruta_archivo, 'r', encoding='utf-8') as f:
                    codigo = f.read()
                    fragmentos_codigo.append(codigo)
                    rutas_archivos.append(ruta_archivo)
    return fragmentos_codigo, rutas_archivos

# Leer y clasificar los archivos del proyecto
fragmentos_codigo, rutas_archivos = leer_archivos_de_directorio(ruta_proyecto)

# Clasificar cada fragmento de código y mostrar la probabilidad
for fragmento, ruta_archivo in zip(fragmentos_codigo, rutas_archivos):
    tactica_predicha, probabilidad_predicha = clasificar_codigo_con_probabilidad(fragmento, modelo, tokenizador, tacticas)
    print(f"Archivo: {ruta_archivo}\nTáctica predicha: {tactica_predicha}\nProbabilidad de predicción: {probabilidad_predicha:.4f}\n")

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-1-b18362ed4524>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explici

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Archivo: /content/drive/MyDrive/proyectos/piggymetrics-master/monitoring/src/test/java/com/piggymetrics/monitoring/MonitoringApplicationTests.java
Táctica predicha: unrelated
Probabilidad de predicción: 0.9975

Archivo: /content/drive/MyDrive/proyectos/piggymetrics-master/monitoring/src/main/java/com/piggymetrics/monitoring/MonitoringApplication.java
Táctica predicha: unrelated
Probabilidad de predicción: 0.9946

Archivo: /content/drive/MyDrive/proyectos/piggymetrics-master/notification-service/src/test/java/com/piggymetrics/notification/NotificationServiceApplicationTests.java
Táctica predicha: unrelated
Probabilidad de predicción: 0.9982

Archivo: /content/drive/MyDrive/proyectos/piggymetrics-master/notification-service/src/test/java/com/piggymetrics/notification/controller/RecipientControllerTest.java
Táctica predicha: token based authentication
Probabilidad de predicción: 0.7992

Archivo: /content/drive/MyDrive/proyectos/piggymetrics-master/notification-service/src/test/java/com/pi